In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brats-2021-task1/BraTS2021_00495.tar
/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar
/kaggle/input/brats-2021-task1/BraTS2021_00621.tar


In [2]:
Train_data_path = '/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar'
!tar -xf /kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar -C /kaggle/working/

In [3]:
!pip install nibabel

In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from glob import glob
import nibabel as nib
from tqdm import tqdm

torch.manual_seed(42)
np.random.seed(42)


In [5]:
BASE = "/kaggle/working"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODALITIES = ["flair", "t1", "t1ce", "t2"]
N_CLASSES = 4
BATCH_SIZE = 8
EPOCHS = 3      # volontairement court pour comparaison
LR = 1e-4


In [6]:
all_dirs = sorted([d for d in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, d))])

valid = []
for p in all_dirs:
    folder = os.path.join(BASE, p)
    has_seg = len(glob(os.path.join(folder, "*seg.nii.gz"))) > 0
    has_mod = any(len(glob(os.path.join(folder, f"*{m}.nii.gz"))) > 0 for m in MODALITIES)
    if has_seg and has_mod:
        valid.append(p)

print("Patients valides:", len(valid))

train_p, val_p = train_test_split(valid, test_size=0.2, random_state=42)
train_paths = [os.path.join(BASE, p) for p in train_p]
val_paths   = [os.path.join(BASE, p) for p in val_p]


Patients valides: 1251


In [7]:
def get_slice_indices(D, mode):
    if mode == "1":
        return [D // 2]
    if mode == "3":
        return [D//4, D//2, 3*D//4]
    if mode == "5":
        return [D//4, 3*D//8, D//2, 5*D//8, 3*D//4]
    if mode == "7":
        return [
            D//4,
            7*D//16,
            3*D//8,
            D//2,
            9*D//16,
            5*D//8,
            3*D//4
        ]
    raise ValueError("Unknown mode")


In [8]:
class BraTS2DDataset(Dataset):
    def __init__(self, patient_paths, mode):
        self.paths = patient_paths
        self.mode = mode

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]

        vols = []
        for m in MODALITIES:
            files = glob(os.path.join(p, f"*{m}.nii.gz"))
            if len(files) == 0:
                vols.append(None)
            else:
                v = nib.load(files[0]).get_fdata().astype(np.float32)
                v = (v - v.mean()) / (v.std() + 1e-5)
                vols.append(v)

        seg = nib.load(glob(os.path.join(p, "*seg.nii.gz"))[0]).get_fdata().astype(np.int64)
        seg[seg == 4] = 3  # 🔴 FIX CRITIQUE CUDA

        D = seg.shape[2]
        slices = get_slice_indices(D, self.mode)

        Xs, Ys = [], []
        for z in slices:
            chans = []
            for v in vols:
                if v is None:
                    chans.append(np.zeros_like(seg[:, :, z], dtype=np.float32))
                else:
                    chans.append(v[:, :, z])
            Xs.append(np.stack(chans, axis=0))
            Ys.append(seg[:, :, z])

        X = torch.from_numpy(np.stack(Xs))
        Y = torch.from_numpy(np.stack(Ys))

        return X, Y


In [9]:
class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.net(x)


class UNet2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.d1 = DoubleConv(4, 32)
        self.d2 = DoubleConv(32, 64)
        self.b  = DoubleConv(64, 128)
        self.u2 = DoubleConv(128+64, 64)
        self.u1 = DoubleConv(64+32, 32)
        self.out = nn.Conv2d(32, N_CLASSES, 1)
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        d1 = self.d1(x)
        d2 = self.d2(self.pool(d1))
        b  = self.b(self.pool(d2))
        u2 = self.u2(torch.cat([nn.functional.interpolate(b, scale_factor=2), d2], 1))
        u1 = self.u1(torch.cat([nn.functional.interpolate(u2, scale_factor=2), d1], 1))
        return self.out(u1)


In [10]:
def dice_score_multiclass(logits, target, eps=1e-6):
    """
    logits: (B, C, H, W)
    target: (B, H, W)
    """
    pred = logits.argmax(1)
    dices = []

    for c in range(1, N_CLASSES):  # ❌ ignore background
        pred_c = (pred == c).float()
        tgt_c  = (target == c).float()

        inter = (pred_c * tgt_c).sum()
        union = pred_c.sum() + tgt_c.sum()

        if union > 0:
            dices.append((2 * inter + eps) / (union + eps))

    if len(dices) == 0:
        return torch.tensor(0.0, device=logits.device)

    return torch.mean(torch.stack(dices))


In [11]:
def train_mode(mode):
    train_ds = BraTS2DDataset(train_paths, mode)
    val_ds   = BraTS2DDataset(val_paths, mode)

    train_dl = DataLoader(train_ds, batch_size=1, shuffle=True)
    val_dl   = DataLoader(val_ds, batch_size=1)

    model = UNet2D().to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss()

    for _ in range(EPOCHS):
        model.train()
        for X, Y in train_dl:
            B,S,C,H,W = X.shape
            X = X.view(B*S, C, H, W).to(DEVICE)
            Y = Y.view(B*S, H, W).to(DEVICE)

            opt.zero_grad()
            loss = loss_fn(model(X), Y)
            loss.backward()
            opt.step()

    model.eval()
    dices = []
    with torch.no_grad():
        for X, Y in val_dl:
            B,S,C,H,W = X.shape
            X = X.view(B*S, C, H, W).to(DEVICE)
            Y = Y.view(B*S, H, W).to(DEVICE)
            logits = model(X)
            dices.append(dice_score_multiclass(logits, Y).item())


    return np.mean(dices)


In [12]:
results = {}
for mode in ["1", "3", "5", "7"]:
    print(f"\nTraining with {mode} slices")
    results[mode] = train_mode(mode)
    print("Val Dice:", results[mode])

print("\n===== RÉSUMÉ =====")
for k,v in results.items():
    print(f"{k} slice(s): Dice = {v:.4f}")



Training with 1 slices
Val Dice: 0.47425273418102215

Training with 3 slices
Val Dice: 0.49159341427430653

Training with 5 slices
Val Dice: 0.5598691744039733

Training with 7 slices
Val Dice: 0.5787818725424934

===== RÉSUMÉ =====
1 slice(s): Dice = 0.4743
3 slice(s): Dice = 0.4916
5 slice(s): Dice = 0.5599
7 slice(s): Dice = 0.5788
